In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

tokenizer = AutoTokenizer.from_pretrained("livekit/turn-detector")

# messages = [
#     {"role": "user", "content": "Who are you?"},
#     {"role": "assistant", "content": "I am John."},
#     {"role": "user", "content": "What is your last name?"},
#     {"role": "assistant", "content": "Smith."},
#     {"role": "user", "content": "How do you spell the first"}
# ]

# messages = [
#     {"role": "user", "content": "Who are you?"},
#     {"role": "assistant", "content": "I am John."},
#     {"role": "user", "content": "What is your last name?"},
#     {"role": "assistant", "content": "Smith."},
# ]


In [27]:
messages = [{
      "timestamp": "00:00:00",
      "role": "Speaker 0",
      "content": "Ma'am, cars. Can I help you?"
    },
    {
      "timestamp": "00:00:02",
      "role": "Speaker 1",
      "content": "Oh, hi. I'm trying to find out how much it would cost for you to pick up something from prior report. It's not it's not furniture or anything like that. Just a a little bag Okay. For my son."
    },
    {
      "timestamp": "00:00:12",
      "role": "Speaker 1",
      "content": "And drop it off at, in washing sorry. Pick it up from Washington House and drop it off at 3 Parkside, Rutland Road, E 97 J U."
    },
    {
      "timestamp": "00:00:28",
      "role": "Speaker 0",
      "content": "20 Pound 10."
    },
    {
      "timestamp": "00:00:30",
      "role": "Speaker 1",
      "content": "Okay. Is it possible to arrange that now?"
    }]

In [83]:
messages = [{
      # "timestamp": "00:00:00",
      # "role": "Speaker 0",
      # "content": "Ma'am, cars. Can I help you?"
    # },
    # {
      # "timestamp": "00:00:02",
      # "role": "Speaker 1",
      # "content": "Oh, hi. I'm trying to find out how much it would cost for you to pick up something from prior report."
    # },
    # {
      # "timestamp": "00:00:12",
      # "role": "Speaker 1",
      # "content": "And drop it off at, in washing sorry. Pick it up from Washington House and drop it off at 3 Parkside, Rutland Road, E 97 J U."
    # },
    # {
      "timestamp": "00:00:28",
      "role": "Speaker 0",
      "content": "20 Pound 10."
    # },
    # {
    #   "timestamp": "00:00:30",
    #   "role": "Speaker 1",
    #   "content": "Okay. Is it possible to arrange that now?"
    }]

In [85]:
# @title
# Format messages using the chat template
text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=False,
    add_special_tokens=False,
    tokenize=False
)

# Remove the EOU token from current utterance
# ix = text.rfind("<|im_end|>")
# text = text[:ix]


In [86]:
text

'<|im_start|><|Speaker 0|>20 Pound 10.<|im_end|>'

In [9]:
# Load model
model = AutoModelForSequenceClassification.from_pretrained(
    "livekit/turn-detector")

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at livekit/turn-detector and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [82]:
model.config

LlamaConfig {
  "_attn_implementation_autoset": true,
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "eos_token_id": 0,
  "head_dim": 64,
  "hidden_act": "silu",
  "hidden_size": 576,
  "initializer_range": 0.041666666666666664,
  "intermediate_size": 1536,
  "is_llama_config": true,
  "max_position_embeddings": 8192,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 9,
  "num_hidden_layers": 30,
  "num_key_value_heads": 3,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_interleaved": false,
  "rope_scaling": null,
  "rope_theta": 100000,
  "tie_word_embeddings": true,
  "torch_dtype": "float32",
  "transformers_version": "4.51.3",
  "use_cache": true,
  "vocab_size": 49154
}

In [88]:
# @title

# Tokenize
inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)

In [89]:
inputs

{'input_ids': tensor([[    1,    44,   108, 15024,  5251,   216,    32,   108,    46,    34,
            32,   377,   620,   216,    33,    32,    30,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [80]:
inputs

{'input_ids': tensor([[    1,    44,   108, 15024,  5251,   216,    32,   108,    46,    34,
            32,   377,   620,   216,    33,    32,    30]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [90]:



# Get prediction
with torch.no_grad():
    outputs = model(**inputs)
    probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)
    print("probabilities", probabilities)
    # Use index 1 for the positive class probability
    eou_probability = probabilities[0, 1].item()

print(f"End of utterance probability: {eou_probability}")

probabilities tensor([[0.4733, 0.5267]])
End of utterance probability: 0.526699960231781


In [27]:
outputs

SequenceClassifierOutputWithPast(loss=None, logits=tensor([[0.8110, 1.0441]]), past_key_values=<transformers.cache_utils.DynamicCache object at 0x7c5ab3ac9650>, hidden_states=None, attentions=None)

Role of Past Key Values
The past_key_values attribute, represented as a DynamicCache object, is crucial for real-time applications.
It caches the key and value states from the transformer model, allowing subsequent inputs to be processed efficiently without recomputing the entire history. This is particularly important for streaming audio, where the model processes input incrementally, as seen in LiveKit’s implementation for EOU detection.



In [13]:
dir(outputs)

['__annotations__',
 '__class__',
 '__class_getitem__',
 '__contains__',
 '__dataclass_fields__',
 '__dataclass_params__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__ior__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__match_args__',
 '__module__',
 '__ne__',
 '__new__',
 '__or__',
 '__post_init__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__ror__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'attentions',
 'clear',
 'copy',
 'fromkeys',
 'get',
 'hidden_states',
 'items',
 'keys',
 'logits',
 'loss',
 'move_to_end',
 'past_key_values',
 'pop',
 'popitem',
 'setdefault',
 'to_tuple',
 'update',
 'values']

In [31]:
outputs.past_key_values[1]

(tensor([[[[ 2.8481e+00,  1.5320e+00, -3.4065e+00,  ..., -7.0517e-01,
            -5.0408e-01, -2.7594e+00],
           [-2.8290e+00, -3.3621e+00, -2.8275e+00,  ...,  1.8865e+00,
            -8.7491e-01,  1.0558e+00],
           [-5.8378e+00, -2.2527e+00, -8.6742e-01,  ...,  9.5778e-01,
             1.6808e-01, -1.6429e+00],
           ...,
           [ 6.5366e+00,  2.3378e+00,  1.8057e+00,  ...,  1.0893e+00,
            -1.4958e+00, -7.6381e-01],
           [ 1.5594e+00,  1.9164e+00,  3.6701e+00,  ...,  2.6238e-01,
            -2.4472e+00, -8.8869e-01],
           [-4.1201e-01,  2.8317e-01,  6.9000e-01,  ..., -7.2752e-02,
            -4.4232e-01, -1.3182e+00]],
 
          [[ 6.1872e-01, -6.3033e-01,  1.0066e+00,  ..., -2.4309e+00,
            -1.0370e+00, -5.1346e-01],
           [ 1.7875e+00, -7.5878e-01, -4.0836e-01,  ..., -7.8726e-01,
             1.6954e+00,  3.6208e-01],
           [-1.5504e-01,  2.5183e+00, -2.1796e+00,  ..., -2.2357e-01,
             9.7780e-01, -5.1814e-01],


In [8]:




# Get prediction
with torch.no_grad():
    outputs = model(**inputs)
    probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)
    print("probabilities", probabilities)
    # Use index 1 for the positive class probability
    eou_probability = probabilities[0, 1].item()

print(f"End of utterance probability: {eou_probability}")

probabilities tensor([[0.8651, 0.1349]])
End of utterance probability: 0.1348775029182434


SequenceClassifierOutputWithPast(loss=None, logits=tensor([[-2.3683, -0.0288]]), past_key_values=<transformers.cache_utils.DynamicCache object at 0x795f75b3c350>, hidden_states=None, attentions=None)

In [16]:
text

"<|im_start|><|Speaker 0|>Ma'am, cars. Can I help you?"